In [108]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from itertools import product
pd.set_option("display.max_rows", 101)
pd.set_option("display.max_columns", 101)

import matplotlib.pyplot as plt
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [109]:
import Metrica_IO as mio
import Metrica_Viz as mviz

In [110]:
# set up initial path to data
DATADIR = os.path.join("sample-data","data")
game_id = 2 # let's look at sample match 2

In [111]:
# read in the event data
events = mio.read_event_data(DATADIR,game_id)

In [112]:
# Bit of housekeeping: unit conversion from metric data units to meters
events = mio.to_metric_coordinates(events)

In [113]:
tracking_home = mio.tracking_data(DATADIR,game_id,'Home')
tracking_away = mio.tracking_data(DATADIR,game_id,'Away')

# Convert positions from metrica units to meters 
tracking_home = mio.to_metric_coordinates(tracking_home)
tracking_away = mio.to_metric_coordinates(tracking_away)

Reading team: home
Reading team: away


In [114]:
passes = events.loc[events["Type"]=="PASS"]
passes.insert(2,"Impact Factor",0)##Set up a column for the impact factor to go into
print(len(passes))

964


In [115]:
def find_pass_indexes(df):##Find the indexes for all the passes since they are not sequential
    index = []
    for i in range(0, 964):
        index.append(df.loc[df['Type'] == 'PASS'].index[i])
    return index
pass_indexes=find_pass_indexes(passes)
print(pass_indexes[963])##there are 964 passes and the last one is the 1929th event, agrees with passes dataframe
print(len(pass_indexes))##

1929
964


In [119]:
threshold=10
for i in range(0,len(pass_indexes)):
    pass_number=pass_indexes[i]##Update: Problem solved
    pass_of_interest_loc_before=passes.loc[pass_number].at["Start Frame"]
    pass_of_interest_team=passes.loc[pass_number].at["Team"]
##Lets get the tracking data from before the pass is made and find how many teammates were open
    if pass_of_interest_team=="Away":
        reciever_teammates_before=tracking_away.loc[pass_of_interest_loc_before][2:26].dropna()##Away has 12 players=>26
        reciever_opponents_before=tracking_home.loc[pass_of_interest_loc_before][2:30].dropna()##Home has 14 players=>30
    else:
        reciever_opponents_before=tracking_away.loc[pass_of_interest_loc_before][2:26].dropna()##Thus these lists will ALWAYS
        reciever_teammates_before=tracking_home.loc[pass_of_interest_loc_before][2:30].dropna()##have 22 entries, GK always in 0,1
    ##Further, we could exclude GK's from this if it makes it better
    teammate_x_before=list(reciever_teammates_before[0:21:2])##Seperate all x and y values and turn into list
    teammate_y_before=list(reciever_teammates_before[1:22:2])##So we can use the zip function to pair up corresponding x values
    opponent_x_before=list(reciever_opponents_before[0:21:2])##made names shorter because it was getting absurd
    opponent_y_before=list(reciever_opponents_before[1:22:2])
    distances_before=[]##initialize the list that will store distances
    for (x1, y1), (x2, y2) in product(zip(teammate_x_before, teammate_y_before), zip(opponent_x_before, opponent_y_before)):
        distance = ((x2 - x1)**2 + (y2 - y1)**2)**0.5
        distances_before.append(distance)
    ##Need to get this so I check if each PLAYER is open
    open_count_before=0##Initialize 
    ##To do this we take 11 elements of distances_before at a time(the distance between the first player in the list and all the player on the other team)
    for i in range(0,11):
        if all(x > threshold for x in distances_before[i*11:(i+1)*11]):##And we check if all those values are greater than a certain number, here: 5
            open_count_before+=1
##Lets get the tracking data from after the pass was made
##This is the same exact process but using the reference of the end frame of the pass rather than the start frame
    pass_of_interest_loc_after=passes.loc[pass_number].at["End Frame"]
    pass_of_interest_team=passes.loc[pass_number].at["Team"]##Should be the same as before##Take this out when compounding code, this is just here as a reminder
    if pass_of_interest_team=="Away":
        reciever_teammates_after=tracking_away.loc[pass_of_interest_loc_after][2:26].dropna()##Away has 12 players=>26
        reciever_opponents_after=tracking_home.loc[pass_of_interest_loc_after][2:30].dropna()##Home has 14 players=>30
    else:
        reciever_opponents_after=tracking_away.loc[pass_of_interest_loc_after][2:26].dropna()##Thus these lists will ALWAYS
        reciever_teammates_after=tracking_home.loc[pass_of_interest_loc_after][2:30].dropna()##have 22 entries, GK always in 0,1
    ##Further, we could exclude GK's from this if it makes it better
##Get the distances for after
    teammate_x_after=list(reciever_teammates_after[0:21:2])
    teammate_y_after=list(reciever_teammates_after[1:22:2])
    opponent_x_after=list(reciever_opponents_after[0:21:2])
    opponent_y_after=list(reciever_opponents_after[1:22:2])
    distances_after=[]##initialize the list
    for (x1,y1), (x2,y2) in product(zip(teammate_x_after,teammate_y_after), zip(opponent_x_after, opponent_y_after)):
        distance = ((x2 - x1)**2 + (y2 - y1)**2)**0.5
        distances_after.append(distance)


    open_count_after=0##Initialize 
##To do this we take 11 elements of distances_before at a time(the distance between the first player in the list and all the player on the other team)

    for i in range(0,11):
        if all(x > threshold for x in distances_after[i*11:(i+1)*11]):##And we check if all those values are greater than a certain number, here: 5
            open_count_after+=1
    iFactor=open_count_after-open_count_before ##Find how many teammates were made open by the pass
    passes.loc[pass_number,('Impact Factor')]=iFactor##Assign that factor to the corresponding row in the data frame

In [120]:
passes

,Team,Type,Impact Factor,Subtype,Period,Start Frame,Start Time [s],End Frame,End Time [s],From,To,Start X,Start Y,End X,End Y
1,Away,PASS,0,NaN,1,51,2.04,87,3.48,Player23,Player20,0.00,-0.00,-10.60,-0.68
2,Away,PASS,1,NaN,1,146,5.84,186,7.44,Player20,Player18,-7.42,-0.00,-6.36,19.04
3,Away,PASS,0,NaN,1,248,9.92,283,11.32,Player18,Player17,-3.18,21.08,-20.14,14.96
4,Away,PASS,0,NaN,1,316,12.64,346,13.84,Player17,Player16,-22.26,12.24,-25.44,-5.44
5,Away,PASS,-1,NaN,1,395,15.80,423,16.92,Player16,Player15,-25.44,-10.88,-24.38,-27.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1914,Home,PASS,-1,NaN,2,137209,5488.36,137221,5488.84,Player1,Player6,-9.54,12.24,-4.24,4.76
1915,Home,PASS,-2,NaN,2,137263,5490.52,137303,5492.12,Player6,Player1,-4.24,12.24,-12.72,21.76
1916,Home,PASS,0,NaN,2,137303,5492.12,137332,5493.28,Player1,Player5,-12.72,21.76,-1.06,26.52
1917,Home,PASS,0,NaN,2,137334,5493.36,137359,5494.36,Player5,Player1,-1.06,26.52,-15.90,26.52


In [125]:
plt.hist(passes)

count    964.000000
mean      -0.029046
std        0.850866
min       -4.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        6.000000
Name: Impact Factor, dtype: float64